In [1]:
#Importing the necessary librarires

import math
import nltk
import scipy
import string
import numpy as np
import pandas as pd
import seaborn as sns

from tqdm import tqdm
from scipy.stats import randint
from wordcloud import WordCloud
from multiprocessing import Pool
from nltk.corpus import stopwords
from scipy.stats import loguniform

from sklearn.decomposition import PCA
from nltk.stem.porter import PorterStemmer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score, \
roc_auc_score, roc_curve, precision_score, recall_score

from IPython.display import display

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10,6]

import warnings 
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'wordcloud'

In [ ]:
#Importing the dataset

tweets_df = pd.read_csv('tweet-sentiment-extraction/train.csv', header=0)

In [ ]:
tweets_df.head()

In [ ]:
tweets_df.describe(include='O')

In [11]:
tweets_df.drop(['selected_text', 'textID'], axis=1, inplace=True)

In [ ]:
tweets_df.head()

In [ ]:
target = 'sentiment'

In [ ]:
original_df = tweets_df.copy(deep=True)

In [ ]:
print('\n\033[1mData Dimension:\033[0m Dataset consists of {} columns & {} records.'.format(tweets_df.shape[1], tweets_df.shape[0]))

In [2]:
#Checking the dtypes of all the columns

tweets_df.info()

NameError: name 'tweets_df' is not defined

In [17]:
#Checking the stats of all the columns

tweets_df.describe()

,text,sentiment
count,27480,27481
unique,27480,3
top,"I`d have responded, if I were going",neutral
freq,1,11118


### 2. Data Preprocessing

In [18]:
#Check for empty elements

tweets_df.isnull().sum()

text         1
sentiment    0
dtype: int64

In [19]:
#Dropping the null values
tweets_df.dropna(inplace=True)
original_df = tweets_df.copy()

In [20]:
tweets_df[tweets_df.duplicated()]

,text,sentiment


In [3]:
#Removal of any Duplicate rows (if any)

counter = 0
r,c = original_df.shape

tweets_df_dedup = tweets_df.drop_duplicates()
tweets_df_dedup.reset_index(drop=True, inplace=True)

if tweets_df_dedup.shape==(r,c):
    print('\n\033[1mInference:\033[0m The dataset doesn\'t have any duplicates')
else:
    print(f'\n\033[1mInference:\033[0m Number of duplicates dropped/fixed ---> {r-tweets_df_dedup.shape[0]}')

NameError: name 'original_df' is not defined